In [153]:
import pandas as pd
import numpy as np

url = 'https://open.canada.ca/data/dataset/1eb9eba7-71d1-4b30-9fb1-30cbdab7e63a/resource/91d586f9-fb72-4eac-b1f5-d9b95830c877/download/y_2019_en.xlsx'

df = pd.read_excel(url)

print(df.head())


   C_YEAR C_MNTH C_WDAY C_HOUR  C_SEV  C_VEHS C_CONF C_RCFG C_WTHR C_RSUR  \
0    2019      1      1     11      2       2     32      1      1      5   
1    2019      1      1     11      2       2     32      1      1      5   
2    2019      1      1     11      2       2     32      1      1      5   
3    2019      1      1     11      2       2     32      1      1      5   
4    2019      1      1     16      2       2     32      1      2      5   

   ... V_TYPE V_YEAR P_ID P_SEX P_AGE P_PSN P_ISEV P_SAFE P_USER   C_CASE  
0  ...      1   2015    1     M    56    QQ      1      2      U  2681651  
1  ...      1   2015   UU     U    UU    UU      U     UU      U  2681651  
2  ...      1   2010    1     F    30    11      2      2      1  2681651  
3  ...      1   2010    2     U     1    QQ      N     NN      U  2681651  
4  ...      1   2007    1     F    27    11      2      2      1  2681652  

[5 rows x 23 columns]


In [154]:
del df['C_YEAR']

In [155]:
column_names = df.columns.tolist()
column_names = [item.replace('P_', '').replace('C_', '').replace('V_', 'Vehicle ')  for item in column_names]

print(column_names)

['MNTH', 'WDAY', 'HOUR', 'SEV', 'VEHS', 'CONF', 'RCFG', 'WTHR', 'RSUR', 'RALN', 'TRAF', 'Vehicle ID', 'Vehicle TYPE', 'Vehicle YEAR', 'ID', 'SEX', 'AGE', 'PSN', 'ISEV', 'SAFE', 'USER', 'CASE']


In [156]:
abbreviations_mapping = {
    'MNTH': 'Month',
    'WDAY': 'Weekday',
    'HOUR': 'Hour',
    'SEV': 'Severity',
    'VEHS': 'Vehicles',
    'CONF': 'Configuration',
    'RCFG': 'Road Configuration',
    'WTHR': 'Weather',
    'RSUR': 'Road Surface',
    'RALN': 'Road Alignment',
    'TRAF': 'Traffic',
    'ID': 'Identifier',
    'SEX': 'Sex',
    'AGE': 'Age',
    'PSN': 'Position',
    'ISEV': 'Injury Severity',
    'SAFE': 'Safety',
    'USER': 'User',
    'CASE': 'Case'
}

new_column_names = [abbreviations_mapping.get(item, item).replace('_', ' ').title() for item in column_names]

print(new_column_names)
df.columns = new_column_names
collision_data = df
collision_data.head()

['Month', 'Weekday', 'Hour', 'Severity', 'Vehicles', 'Configuration', 'Road Configuration', 'Weather', 'Road Surface', 'Road Alignment', 'Traffic', 'Vehicle Id', 'Vehicle Type', 'Vehicle Year', 'Identifier', 'Sex', 'Age', 'Position', 'Injury Severity', 'Safety', 'User', 'Case']


,Month,Weekday,Hour,Severity,Vehicles,Configuration,Road Configuration,Weather,Road Surface,Road Alignment,...,Vehicle Type,Vehicle Year,Identifier,Sex,Age,Position,Injury Severity,Safety,User,Case
0,1,1,11,2,2,32,1,1,5,4,...,1,2015,1,M,56,QQ,1,2,U,2681651
1,1,1,11,2,2,32,1,1,5,4,...,1,2015,UU,U,UU,UU,U,UU,U,2681651
2,1,1,11,2,2,32,1,1,5,4,...,1,2010,1,F,30,11,2,2,1,2681651
3,1,1,11,2,2,32,1,1,5,4,...,1,2010,2,U,1,QQ,N,NN,U,2681651
4,1,1,16,2,2,32,1,2,5,3,...,1,2007,1,F,27,11,2,2,1,2681652


In [157]:
pip install tabula-py

In [158]:
import tabula
from google.colab import drive
drive.mount('/content/drive/')

pdf_path = '/content/drive/MyDrive/Colab Notebooks/dictionary.pdf'
dict_tables = tabula.read_pdf(pdf_path, pages='1-10')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Jul 18, 2023 11:24:24 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jul 18, 2023 11:24:28 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jul 18, 2023 11:24:28 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [159]:
dict_list = []

for i in range(len(dict_tables)):
  df = pd.DataFrame(dict_tables[i])
  dict_list.append(df)

In [160]:
for i in range(len(dict_list)):
  dict_list[i] = dict_list[i].replace('NaN', np.nan)

for i in range(len(dict_list)):
  dict_list[i] = dict_list[i].dropna(axis=1, how='all')


In [161]:
dict_list = [df for df in dict_list if not df.empty]


In [162]:
del dict_list[0]

In [163]:
len(dict_list), len(new_column_names)

(21, 22)

In [164]:
dictionary = {new_column_names[i]: df.iloc[1:].reset_index(drop=True).rename(columns=df.iloc[0]) for i, df in enumerate(dict_list)}

In [165]:
def find_description(category, row_num:int):
  table = dictionary[category]
  lookup = collision_data.iloc[row_num][category]
  description = table.loc[table['Code'] == lookup, 'Description'].values[0]
  return description

In [166]:
find_description('Sex', 5)

'Female'

In [167]:
def categorize_time(hour):

    try:
        hour = int(hour)
    except ValueError:
        return np.nan
    if hour < 6 or hour >= 20:
        return 'Night'
    elif hour < 12:
        return 'Morning'
    elif hour < 17:
        return 'Afternoon'
    else:
        return 'Evening'


collision_data['Time of Day'] = collision_data['Hour'].apply(categorize_time)
collision_data.head()

,Month,Weekday,Hour,Severity,Vehicles,Configuration,Road Configuration,Weather,Road Surface,Road Alignment,...,Vehicle Year,Identifier,Sex,Age,Position,Injury Severity,Safety,User,Case,Time of Day
0,1,1,11,2,2,32,1,1,5,4,...,2015,1,M,56,QQ,1,2,U,2681651,Morning
1,1,1,11,2,2,32,1,1,5,4,...,2015,UU,U,UU,UU,U,UU,U,2681651,Morning
2,1,1,11,2,2,32,1,1,5,4,...,2010,1,F,30,11,2,2,1,2681651,Morning
3,1,1,11,2,2,32,1,1,5,4,...,2010,2,U,1,QQ,N,NN,U,2681651,Morning
4,1,1,16,2,2,32,1,2,5,3,...,2007,1,F,27,11,2,2,1,2681652,Afternoon


In [168]:
columns_to_convert = collision_data.columns[:-1]
columns_to_convert = [col for col in columns_to_convert if col != 'Sex']
columns_to_convert

['Month',
 'Weekday',
 'Hour',
 'Severity',
 'Vehicles',
 'Configuration',
 'Road Configuration',
 'Weather',
 'Road Surface',
 'Road Alignment',
 'Traffic',
 'Vehicle Id',
 'Vehicle Type',
 'Vehicle Year',
 'Identifier',
 'Age',
 'Position',
 'Injury Severity',
 'Safety',
 'User',
 'Case']

In [169]:


def convert_to_numeric(value):
    try:
        return int(value)
    except ValueError:
        return np.nan

# Apply the conversion function to selected columns
collision_data[columns_to_convert] = collision_data[columns_to_convert].applymap(convert_to_numeric)
collision_data[columns_to_convert] = collision_data[columns_to_convert].astype('Int64')

In [170]:

collision_data.head()

,Month,Weekday,Hour,Severity,Vehicles,Configuration,Road Configuration,Weather,Road Surface,Road Alignment,...,Vehicle Year,Identifier,Sex,Age,Position,Injury Severity,Safety,User,Case,Time of Day
0,1,1,11,2,2,32,1,1,5,4,...,2015,1,M,56,<NA>,1,2,<NA>,2681651,Morning
1,1,1,11,2,2,32,1,1,5,4,...,2015,<NA>,U,<NA>,<NA>,<NA>,<NA>,<NA>,2681651,Morning
2,1,1,11,2,2,32,1,1,5,4,...,2010,1,F,30,11,2,2,1,2681651,Morning
3,1,1,11,2,2,32,1,1,5,4,...,2010,2,U,1,<NA>,<NA>,<NA>,<NA>,2681651,Morning
4,1,1,16,2,2,32,1,2,5,3,...,2007,1,F,27,11,2,2,1,2681652,Afternoon


In [171]:
dictionary['Weekday'].columns = dictionary['Weekday'].iloc[0]
dictionary['Weekday'] = pd.DataFrame(dictionary['Weekday'].values[1:], columns=['Code', 'Description'])

dictionary['Month'].columns = dictionary['Month'].iloc[0]
dictionary['Month'] = pd.DataFrame(dictionary['Month'].values[1:], columns=['Code', 'Description'])

dictionary['Hour'].columns = dictionary['Hour'].iloc[0]
dictionary['Hour'] = pd.DataFrame(dictionary['Hour'].values[1:], columns=['Code', 'Description'])

print(dictionary['Weekday'])
print(dictionary['Month'])
print(dictionary['Hour'])

  Code                                      Description
0    3                                        Wednesday
1    4                                         Thursday
2    5                                           Friday
3    6                                         Saturday
4    7                                           Sunday
5    U                                          Unknown
6    X  Jurisdiction does not provide this data element
   Code                                      Description
0    03                                            March
1    04                                            April
2    05                                              May
3    06                                             June
4    07                                             July
5    08                                           August
6    09                                        September
7    10                                          October
8    11                                

In [172]:
column = collision_data.columns[0]
dictionary[column]

new_rows = [['01', 'January'], ['02', 'February']]
new_rows_df = pd.DataFrame(new_rows, columns=dictionary[column].columns)

# Concatenate the new rows DataFrame with the original DataFrame
dictionary[column] = pd.concat([new_rows_df, dictionary[column]], ignore_index=True)

# Print the updated DataFrame


In [173]:
column = collision_data.columns[1]
dictionary[column]

new_rows = [['1', 'Monday'], ['2', 'Tuesday']]
new_rows_df = pd.DataFrame(new_rows, columns=dictionary[column].columns)

# Concatenate the new rows DataFrame with the original DataFrame
dictionary[column] = pd.concat([new_rows_df, dictionary[column]], ignore_index=True)

In [174]:
column = collision_data.columns[2]
dictionary[column]

new_rows = [['0', '0:00 to 0:59'], ['01', '1:00 to 1:59']]
new_rows_df = pd.DataFrame(new_rows, columns=dictionary[column].columns)

# Concatenate the new rows DataFrame with the original DataFrame
dictionary[column] = pd.concat([new_rows_df, dictionary[column]], ignore_index=True)

In [175]:
for key, df in dictionary.items():
    # Attempt to convert the first column to integers:
    if key != 'Sex':
      try:
          df['Code'] = pd.to_numeric(df['Code'], errors='coerce', downcast='integer')
      except ValueError:
          df['Code'] = np.nan

for key, df in dictionary.items():
    print(f"DataFrame: {key}")
    print(df)
    print()

DataFrame: Month
    Code                                      Description
0    1.0                                          January
1    2.0                                         February
2    3.0                                            March
3    4.0                                            April
4    5.0                                              May
5    6.0                                             June
6    7.0                                             July
7    8.0                                           August
8    9.0                                        September
9   10.0                                          October
10  11.0                                         November
11  12.0                                         December
12   NaN                                          Unknown
13   NaN  Jurisdiction does not provide this data element

DataFrame: Weekday
   Code                                      Description
0   1.0                             

In [176]:
processed_df = pd.DataFrame()

columns_to_change = ['Month', 'Weekday', 'Severity',
                     'Configuration', 'Road Configuration', 'Weather',
                     'Road Surface', 'Road Alignment', 'Traffic', 'Vehicle Type',
                     'Sex', 'Position', 'Injury Severity',
                     'Safety', 'User']



for column in collision_data.columns:
  if column in columns_to_change:
    code_desc_df = dictionary[column]
    code_desc_df = code_desc_df.drop_duplicates(subset="Code", keep="first")
    code_to_desc_mapping = dict(zip(code_desc_df['Code'], code_desc_df['Description']))

    collision_df_reset_index = collision_data.reset_index(drop=True)
    processed_column = collision_df_reset_index[column].map(code_to_desc_mapping)

    processed_df[column] = processed_column

  else:
    processed_df[column] = collision_data[column]


In [177]:
print(processed_df.head())

     Month Weekday  Hour                              Severity  Vehicles  \
0  January  Monday    11  Collision producing non-fatal injury         2   
1  January  Monday    11  Collision producing non-fatal injury         2   
2  January  Monday    11  Collision producing non-fatal injury         2   
3  January  Monday    11  Collision producing non-fatal injury         2   
4  January  Monday    16  Collision producing non-fatal injury         2   

            Configuration                Road Configuration  \
0  Approaching side-swipe  Non-intersectione.g. 'mid-block'   
1  Approaching side-swipe  Non-intersectione.g. 'mid-block'   
2  Approaching side-swipe  Non-intersectione.g. 'mid-block'   
3  Approaching side-swipe  Non-intersectione.g. 'mid-block'   
4  Approaching side-swipe  Non-intersectione.g. 'mid-block'   

                                 Weather             Road Surface  \
0                        Clear and sunny  IcyIncludes packed snow   
1                        C

In [178]:
#Commented out for now, reactivate if u need to donwloand as excel
# !pip install openpyxl

# Save the DataFrame as an Excel file
#excel_file_path = '/content/processed_data.xlsx'  # Specify the file path and name
#processed_df.to_excel(excel_file_path, index=False)

# Download the Excel file
#from google.colab import files
#files.download(excel_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>